In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix


In [34]:
game_data = pd.read_csv('D1_2015_Processed_Stats.csv')
print("Number of records= ", game_data.shape[0])
game_data.info()

Number of records=  5149
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5149 entries, 0 to 5148
Data columns (total 15 columns):
game_id                      5149 non-null object
game_date                    5149 non-null object
principal_team               5149 non-null object
opponent_team                5149 non-null object
home_team                    5149 non-null bool
principal_score              5149 non-null int64
opponent_score               5149 non-null int64
delta_avg_points_per_game    5149 non-null float64
delta_field_goals_pct        5149 non-null float64
delta_avg_off_rebounds       5149 non-null float64
delta_avg_free_throws_att    5149 non-null float64
delta_avg_free_throws_pct    5149 non-null float64
delta_avg_turnovers          5149 non-null float64
delta_win_pct                5149 non-null float64
game_result                  5149 non-null object
dtypes: bool(1), float64(7), int64(2), object(5)
memory usage: 568.3+ KB


In [35]:
numeric_feature_to_scale = ['delta_avg_points_per_game','delta_avg_off_rebounds','delta_avg_free_throws_att','delta_avg_turnovers']

game_data_scale = game_data[numeric_feature_to_scale].copy()
scaler =StandardScaler()
scaled_features = scaler.fit_transform(game_data_scale.values)
scaled_df = pd.DataFrame(scaled_features, index=game_data_scale.index, columns=game_data_scale.columns)

# Add the categorical column 
scaled_df['home_team'] = game_data['home_team']
scaled_df['delta_field_goals_pct'] = game_data['delta_field_goals_pct']
scaled_df['delta_avg_free_throws_pct'] = game_data['delta_avg_free_throws_pct']

scaled_df['delta_win_pct'] = game_data['delta_win_pct']

scaled_df.drop(columns=['delta_avg_points_per_game', 'delta_avg_free_throws_att'],inplace=True)
scaled_df.head()

,delta_avg_off_rebounds,delta_avg_turnovers,home_team,delta_field_goals_pct,delta_avg_free_throws_pct,delta_win_pct
0,-1.091424,0.812673,False,0.075,-0.008,0.0
1,-1.814260,0.405574,True,-0.041,0.111,0.0
2,-2.537095,-2.037022,True,-0.052,0.276,1.0
3,3.064879,1.626872,True,0.030,-0.144,0.0
4,-0.368589,-6.922215,True,0.011,-0.005,1.0


In [36]:
X =scaled_df
X = pd.get_dummies(X, columns=['home_team'], drop_first=True)
y= game_data['game_result'].copy()
y = pd.get_dummies(y, columns=['game_result'], drop_first=True)
y=y.WIN
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 10)

mlp = MLPClassifier(hidden_layer_sizes=(8,16,8),max_iter=1000)
mlp.fit(X_train,y_train)


In [40]:
y_pred = mlp.predict(X_test)
print("Neural Net model accuracy is %2.2f" % metrics.accuracy_score(y_test, y_pred))

Neural Net model accuracy is 0.70


In [42]:
print(classification_report(y_test,y_pred))  

             precision    recall  f1-score   support

          0       0.71      0.71      0.71       540
          1       0.68      0.68      0.68       490

avg / total       0.70      0.70      0.70      1030

